In [34]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import model_utils

import polars as pl
# import data
def load_data():
    balltoball = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "balltoball.csv")))
    team_stats = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "team12_stats.csv")))
    players_stats = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "players_stats.csv")))
    return balltoball, team_stats, players_stats
balltoball,team_stats,players_stats = load_data()
print(balltoball.columns)
print(team_stats.columns)
print(players_stats.columns)
print(balltoball.head(1),team_stats.head(1),players_stats.head(1))

['match_id', 'innings', 'ball', 'runs', 'wickets', 'curr_score', 'curr_wickets', 'overs', 'run_rate', 'required_run_rate', 'target', 'won']
['match_id', 'gender', 'Cumulative Won team1', 'Cumulative Lost team1', 'Cumulative Tied team1', 'Cumulative W/L team1', 'Cumulative AveRPW team1', 'Cumulative AveRPO team1', 'Cumulative Won team2', 'Cumulative Lost team2', 'Cumulative Tied team2', 'Cumulative W/L team2', 'Cumulative AveRPW team2', 'Cumulative AveRPO team2']
['match_id', 'Cum Mat Total', 'Cum Runs Total', 'Cum SR', 'Cumulative Overs', 'Cumulative Bowling Runs', 'Cumulative Wkts', 'Cumulative Econ', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I']
shape: (1, 12)
┌──────────┬─────────┬──────┬──────┬───┬──────────┬───────────────────┬────────┬─────┐
│ match_id ┆ innings ┆ ball ┆ runs ┆ … ┆ run_rate ┆ required_run_rate ┆ target ┆ won │
│ ---      ┆ ---     ┆ ---  ┆ ---  ┆   ┆ ---      ┆ ---               ┆ ---    ┆ --- │
│ i64      ┆ i64     ┆ f64  ┆ i64  ┆   ┆ f64

In [35]:
def partition_data_with_keys(df, group_keys):
    partitions = df.partition_by(group_keys)
    keys = [tuple(partition.select(group_keys).unique().to_numpy()[0]) for partition in partitions]
    partitions = [partition.drop(group_keys).to_numpy() for partition in partitions]
    # partitions = [partition for partition in partitions]                  # for testing
    return keys, partitions

# Use the updated partition_data_with_keys function
balltoball_keys, balltoball_partitions = partition_data_with_keys(balltoball, ["match_id"])
team_stats_keys, team_stats_partitions = partition_data_with_keys(team_stats, ["match_id"])
players_stats_keys, players_stats_partitions = partition_data_with_keys(players_stats, ["match_id"])

# Align the partitions using common keys
common_keys = set(balltoball_keys) & set(team_stats_keys) & set(players_stats_keys)

balltoball_dict = dict(zip(balltoball_keys, balltoball_partitions))
team_stats_dict = dict(zip(team_stats_keys, team_stats_partitions))
players_stats_dict = dict(zip(players_stats_keys, players_stats_partitions))

aligned_balltoball_partitions = []
aligned_team_stats_partitions = []
aligned_players_stats_partitions = []
labels = []

for key in common_keys:
    balltoball_partition = balltoball_dict[key]
    team_stats_partition = team_stats_dict[key]
    players_stats_partition = players_stats_dict[key]

    label = balltoball_partition[:, -1][0]
    aligned_balltoball_partitions.append(balltoball_partition[:, :-1])
    aligned_team_stats_partitions.append(team_stats_partition)
    aligned_players_stats_partitions.append(players_stats_partition)
    labels.append(label)

import numpy as np
labels = np.array(labels)

In [36]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

team_data = [team.to_numpy() if isinstance(team, pl.DataFrame) else team for team in aligned_team_stats_partitions]
player_data = [players.to_numpy() if isinstance(players, pl.DataFrame) else players for players in aligned_players_stats_partitions]
ball_data = [ball.to_numpy() if isinstance(ball, pl.DataFrame) else ball for ball in aligned_balltoball_partitions]

train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, random_state=42)
val_indices, test_indices = train_test_split(val_indices, test_size=0.5, random_state=42)

dataset = model_utils.CricketDataset(
    team_data,
    player_data,
    ball_data,
    labels
)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)


train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=model_utils.collate_fn_with_packing)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=model_utils.collate_fn_with_packing)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=model_utils.collate_fn_with_packing)

In [37]:
import pickle
with open(os.path.join( '..',"data", "pytorch_data" , "train_dataloader.pkl"), "wb") as f:
    pickle.dump(train_dataloader, f)

with open(os.path.join( '..',"data", "pytorch_data" , "val_dataloader.pkl"), "wb") as f:
    pickle.dump(val_dataloader, f)

with open(os.path.join( '..',"data", "pytorch_data" , "test_dataloader.pkl"), "wb") as f:
    pickle.dump(test_dataloader, f)

In [38]:
train_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "train_dataloader.pkl"), "rb"))
val_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "val_dataloader.pkl"), "rb"))
test_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "test_dataloader.pkl"), "rb"))

In [39]:
j=0
for i in train_dataloader:
    j+=1
print(j)

26


In [49]:
# Print a single match's data
single_match_index = 2  # Index of the match to print
single_match_data = dataset[single_match_index]

team_input, player_input, ball_input, label = single_match_data

team_df = pl.DataFrame(team_input.numpy().reshape(1, -1), schema=team_stats.columns[1:])
player_df = pl.DataFrame(player_input.numpy(), schema=players_stats.columns[1:])
ball_df = pl.DataFrame(ball_input.numpy(), schema=balltoball.columns[1:-1])

print("Single Match Data:")
print("Team Stats:")
team_df

Single Match Data:
Team Stats:


gender,Cumulative Won team1,Cumulative Lost team1,Cumulative Tied team1,Cumulative W/L team1,Cumulative AveRPW team1,Cumulative AveRPO team1,Cumulative Won team2,Cumulative Lost team2,Cumulative Tied team2,Cumulative W/L team2,Cumulative AveRPW team2,Cumulative AveRPO team2
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.0,24.0,19.0,1.0,1.26,27.209999,7.99,44.0,25.0,2.0,1.76,22.98,7.45


In [50]:
print("Player Stats:")
player_df

Player Stats:


Cum Mat Total,Cum Runs Total,Cum SR,Cumulative Overs,Cumulative Bowling Runs,Cumulative Wkts,Cumulative Econ,Cumulative Dis,Cumulative Ct,Cumulative St,Cumulative D/I
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
35.0,531.0,120.93,11.2,113.0,1.0,10.98,15.0,15.0,0.0,0.43
1.0,5.0,45.450001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,558.0,125.980003,20.4,159.0,3.0,7.86,8.0,8.0,0.0,0.4
33.0,791.0,147.369995,52.400002,373.0,23.0,7.34,8.0,8.0,0.0,0.25
36.0,840.0,128.029999,16.0,141.0,5.0,8.69,16.0,16.0,0.0,0.44
42.0,748.0,114.839996,0.0,0.0,0.0,0.0,29.0,21.0,8.0,0.71
18.0,53.0,132.740005,70.5,530.0,14.0,7.49,3.0,3.0,0.0,0.17
14.0,74.0,78.389999,45.400002,331.0,7.0,7.36,6.0,6.0,0.0,0.43
1.0,0.0,0.0,4.0,21.0,1.0,5.25,0.0,0.0,0.0,0.0


In [52]:
print("Label:")
print(label)
print("Ball-to-Ball Stats:")
ball_df.with_row_index()

Label:
tensor(0.)
Ball-to-Ball Stats:


index,innings,ball,runs,wickets,curr_score,curr_wickets,overs,run_rate,required_run_rate,target
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.3,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.4,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.6,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.1,1.0,0.0,5.0,0.0,1.0,5.0,0.0,0.0
7,1.0,1.2,0.0,0.0,5.0,0.0,1.0,5.0,0.0,0.0
8,1.0,1.3,0.0,0.0,5.0,0.0,1.0,5.0,0.0,0.0
